## Agosto

In [51]:
ag95 = spark.read.text("C:/Users/nora.hafidi/Desktop/Big Data/NASA/NASA_access_log_Aug95/access_log_Aug95")

In [15]:
ag95.show(3, False)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
+--------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [1]:
expresion = """(.*)[\s-]{4}\s\[(\d\d)\/.*\/1995:(\d\d):(\d\d):(\d\d).*\s\"\s?([A-Z]*)\s(\/\S*).*?(HTTP.*)?\"\s?(\d\d\d)\s(\d*)"""

In [23]:
from pyspark.sql.functions import regexp_extract
ag95_prima = ag95.select(regexp_extract("value", expresion, 1).alias("host"),
                         regexp_extract("value", expresion, 2).alias("day"),
                         regexp_extract("value", expresion, 3).alias("hours"),
                         regexp_extract("value", expresion, 4).alias("minutes"),
                         regexp_extract("value", expresion, 5).alias("seconds"),
                         regexp_extract("value", expresion, 6).alias("request"),
                         regexp_extract("value", expresion, 7).alias("resource"),
                         regexp_extract("value", expresion, 8).alias("protocol"),
                         regexp_extract("value", expresion, 9).alias("http_status_code"),
                         regexp_extract("value", expresion, 10).alias("size"),         
                        )

In [24]:
ag95_prima.show()

+--------------------+---+-----+-------+-------+-------+--------------------+--------+----------------+-----+
|                host|day|hours|minutes|seconds|request|            resource|protocol|http_status_code| size|
+--------------------+---+-----+-------+-------+-------+--------------------+--------+----------------+-----+
|   in24.inetnebr.com| 01|   00|     00|     01|    GET|/shuttle/missions...|HTTP/1.0|             200| 1839|
|     uplherc.upl.com| 01|   00|     00|     07|    GET|                   /|HTTP/1.0|             304|    0|
|     uplherc.upl.com| 01|   00|     00|     08|    GET|/images/ksclogo-m...|HTTP/1.0|             304|    0|
|     uplherc.upl.com| 01|   00|     00|     08|    GET|/images/MOSAIC-lo...|HTTP/1.0|             304|    0|
|     uplherc.upl.com| 01|   00|     00|     08|    GET|/images/USA-logos...|HTTP/1.0|             304|    0|
|ix-esc-ca2-07.ix....| 01|   00|     00|     09|    GET|/images/launch-lo...|HTTP/1.0|             200| 1713|
|     uplh

In [47]:
nulls_h = (ag95_prima
.select("http_status_code")
.where(col("http_status_code") == '')
.count())
nulls_h

67

In [40]:
cons1 = (ag95_prima
.select("protocol")
.where(col("protocol") != "")
.groupBy("protocol")
.count())
#cons1.show(False)
cons1.show()

+--------------------+-------+
|            protocol|  count|
+--------------------+-------+
|HTTP/1.0From:  <b...|    624|
|           HTTP/1.0 |    113|
|           HTTP/V1.0|    163|
|            HTTP/1.0|1567056|
+--------------------+-------+



In [44]:
cons2 = (ag95_prima
.select("http_status_code")
.groupBy("http_status_code")
.count()
.orderBy("count", descending=False))
cons2.show()

+----------------+-------+
|http_status_code|  count|
+----------------+-------+
|             500|      3|
|             501|     27|
|                |     67|
|             403|    171|
|             404|  10052|
|             302|  26444|
|             304| 134146|
|             200|1398988|
+----------------+-------+



In [45]:
cons3 = (ag95_prima
.select("request")
.groupBy("request")
.count()
.orderBy("count", descending=False))
cons3.show()

+-------+-------+
|request|  count|
+-------+-------+
|       |     67|
|   POST|    111|
|   HEAD|   3965|
|    GET|1565755|
+-------+-------+



In [53]:
cons4 = (ag95_prima
.groupBy("resource")
.count()
.orderBy("count", descending=False))
cons4.show(1, False)

+------------------------------------+-----+
|resource                            |count|
+------------------------------------+-----+
|/cgi-bin/imagemap/countdown70?75,222|1    |
+------------------------------------+-----+
only showing top 1 row



In [54]:
cons5 = (ag95_prima
.orderBy("size", descending=False))
cons5.show(1, False)

+----------------+---+-----+-------+-------+-------+-----------------+--------+----------------+----+
|host            |day|hours|minutes|seconds|request|resource         |protocol|http_status_code|size|
+----------------+---+-----+-------+-------+-------+-----------------+--------+----------------+----+
|jsanz.agr.upv.es|30 |06   |23     |29     |GET    |/shuttle/missions|HTTP/1.0|302             |    |
+----------------+---+-----+-------+-------+-------+-----------------+--------+----------------+----+
only showing top 1 row



In [55]:
cons6 = (ag95_prima
.select("day")
.groupBy("day")
.count()
.orderBy("count", descending=False))
cons6.show(3, False)

+---+-----+
|day|count|
+---+-----+
|   |67   |
|26 |31608|
|05 |31893|
+---+-----+
only showing top 3 rows



In [56]:
cons7 = (ag95_prima
.select("host")
.groupBy("host")
.count()
.orderBy("count", descending=False))
cons7.show(5, False)

+----------------------------+-----+
|host                        |count|
+----------------------------+-----+
|dialup-69.werple.mira.net.au|1    |
|paul.hip.cam.org            |1    |
|hopsden.lib.csufresno.edu   |1    |
|140.90.23.106               |1    |
|cat30.slis.uwo.ca           |1    |
+----------------------------+-----+
only showing top 5 rows



In [57]:
cons8 = (ag95_prima
.select("hours")
.groupBy("hours")
.count()
.orderBy("count", descending=False))
cons8.show(5, False)

+-----+-----+
|hours|count|
+-----+-----+
|     |67   |
|04   |26755|
|05   |27586|
|03   |29993|
|06   |31287|
+-----+-----+
only showing top 5 rows



In [66]:
cons9 = (ag95_prima
.select("day")
.where(col("http_status_code") == "404")
.groupBy("day")
.count()
.orderBy("day"))
cons9.show(31)

+---+-----+
|day|count|
+---+-----+
| 01|  243|
| 03|  304|
| 04|  344|
| 05|  236|
| 06|  373|
| 07|  537|
| 08|  391|
| 09|  279|
| 10|  315|
| 11|  263|
| 12|  196|
| 13|  216|
| 14|  287|
| 15|  327|
| 16|  259|
| 17|  270|
| 18|  256|
| 19|  208|
| 20|  312|
| 21|  305|
| 22|  288|
| 23|  345|
| 24|  420|
| 25|  415|
| 26|  366|
| 27|  370|
| 28|  410|
| 29|  420|
| 30|  571|
| 31|  526|
+---+-----+



## Julio

In [49]:
jul95 = spark.read.text("C:/Users/nora.hafidi/Desktop/Big Data/NASA/NASA_access_log_Jul95/access_log_Jul95")

In [50]:
jul95.show(3, False)

+--------------------------------------------------------------------------------------------------------------------+
|value                                                                                                               |
+--------------------------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245                              |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985                   |
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085|
+--------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [60]:
from pyspark.sql.functions import regexp_extract
jul95_prima = jul95.select(regexp_extract("value", expresion, 1).alias("host"),
                         regexp_extract("value", expresion, 2).alias("day"),
                         regexp_extract("value", expresion, 3).alias("hours"),
                         regexp_extract("value", expresion, 4).alias("minutes"),
                         regexp_extract("value", expresion, 5).alias("seconds"),
                         regexp_extract("value", expresion, 6).alias("request"),
                         regexp_extract("value", expresion, 7).alias("resource"),
                         regexp_extract("value", expresion, 8).alias("protocol"),
                         regexp_extract("value", expresion, 9).alias("http_status_code"),
                         regexp_extract("value", expresion, 10).alias("size"),         
                        )

In [61]:
jul95_prima.show()

+--------------------+---+-----+-------+-------+-------+--------------------+--------+----------------+-----+
|                host|day|hours|minutes|seconds|request|            resource|protocol|http_status_code| size|
+--------------------+---+-----+-------+-------+-------+--------------------+--------+----------------+-----+
|        199.72.81.55| 01|   00|     00|     01|    GET|    /history/apollo/|HTTP/1.0|             200| 6245|
|unicomp6.unicomp.net| 01|   00|     00|     06|    GET| /shuttle/countdown/|HTTP/1.0|             200| 3985|
|      199.120.110.21| 01|   00|     00|     09|    GET|/shuttle/missions...|HTTP/1.0|             200| 4085|
|  burger.letters.com| 01|   00|     00|     11|    GET|/shuttle/countdow...|HTTP/1.0|             304|    0|
|      199.120.110.21| 01|   00|     00|     11|    GET|/shuttle/missions...|HTTP/1.0|             200| 4179|
|  burger.letters.com| 01|   00|     00|     12|    GET|/images/NASA-logo...|HTTP/1.0|             304|    0|
|  burger.

In [62]:
nulls_h_j = (jul95_prima
.select("http_status_code")
.where(col("http_status_code") == '')
.count())
nulls_h_j

13

In [68]:
cons1_j = (jul95_prima
.select("protocol")
.where(col("protocol") != "")
.groupBy("protocol")
.count())
#cons1.show(False)
cons1_j.show()

+--------------------+-------+
|            protocol|  count|
+--------------------+-------+
|HTTP/1.0From:  <b...|    611|
|              HTTP/*|     13|
|           HTTP/1.0 |     15|
|           HTTP/V1.0|    116|
|            HTTP/1.0|1887956|
+--------------------+-------+



In [69]:
cons2_j = (jul95_prima
.select("http_status_code")
.groupBy("http_status_code")
.count()
.orderBy("count", descending=False))
cons2_j.show()

+----------------+-------+
|http_status_code|  count|
+----------------+-------+
|                |     13|
|             501|     14|
|             403|     54|
|             500|     62|
|             404|  10840|
|             302|  46571|
|             304| 132627|
|             200|1701534|
+----------------+-------+



In [67]:
cons3_j = (jul95_prima
.select("request")
.groupBy("request")
.count()
.orderBy("count", descending=False))
cons3_j.show()

+-------+-------+
|request|  count|
+-------+-------+
|       |     13|
|   POST|    111|
|   HEAD|   3952|
|    GET|1887639|
+-------+-------+



In [70]:
cons4_j = (jul95_prima
.groupBy("resource")
.count()
.orderBy("count", descending=False))
cons4_j.show(1, False)

+-----------------------------------+-----+
|resource                           |count|
+-----------------------------------+-----+
|/cgi-bin/imagemap/astrohome?247,271|1    |
+-----------------------------------+-----+
only showing top 1 row



In [72]:
cons5_j = (jul95_prima
.orderBy("size", descending=False))
cons5_j.show(1, False)

+-------------+---+-----+-------+-------+-------+---------------------+--------+----------------+----+
|host         |day|hours|minutes|seconds|request|resource             |protocol|http_status_code|size|
+-------------+---+-----+-------+-------+-------+---------------------+--------+----------------+----+
|malibu.sfu.ca|25 |04   |43     |26     |GET    |/pub/winvn/readme.txt|HTTP/1.0|404             |    |
+-------------+---+-----+-------+-------+-------+---------------------+--------+----------------+----+
only showing top 1 row



In [73]:
cons6_j = (jul95_prima
.select("day")
.groupBy("day")
.count()
.orderBy("count", descending=False))
cons6_j.show(3, False)

+---+-----+
|day|count|
+---+-----+
|   |13   |
|28 |27121|
|22 |35267|
+---+-----+
only showing top 3 rows



In [74]:
cons7_j = (jul95_prima
.select("host")
.groupBy("host")
.count()
.orderBy("count", descending=False))
cons7_j.show(5, False)

+----------------------------+-----+
|host                        |count|
+----------------------------+-----+
|137.82.222.221              |1    |
|flow.infotech.tu-chemnitz.de|1    |
|202.49.243.63               |1    |
|owl.ins.cwru.edu            |1    |
|ppp-103.magg.net            |1    |
+----------------------------+-----+
only showing top 5 rows



In [75]:
cons8_j = (jul95_prima
.select("hours")
.groupBy("hours")
.count()
.orderBy("count", descending=False))
cons8_j.show(5, False)

+-----+-----+
|hours|count|
+-----+-----+
|     |13   |
|05   |31919|
|04   |32234|
|06   |35253|
|03   |37398|
+-----+-----+
only showing top 5 rows



In [76]:
cons9_j = (jul95_prima
.select("day")
.where(col("http_status_code") == "404")
.groupBy("day")
.count()
.orderBy("day"))
cons9_j.show(31)

+---+-----+
|day|count|
+---+-----+
| 01|  315|
| 02|  290|
| 03|  474|
| 04|  359|
| 05|  497|
| 06|  640|
| 07|  570|
| 08|  302|
| 09|  348|
| 10|  398|
| 11|  471|
| 12|  470|
| 13|  531|
| 14|  413|
| 15|  254|
| 16|  257|
| 17|  406|
| 18|  465|
| 19|  639|
| 20|  428|
| 21|  334|
| 22|  192|
| 23|  233|
| 24|  328|
| 25|  461|
| 26|  335|
| 27|  336|
| 28|   94|
+---+-----+

